In [40]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
from sklearn.impute import SimpleImputer
from datetime import timedelta

north_data = pd.read_csv('./weather-csv/north_weekly_averages.csv')
south_data = pd.read_csv('./weather-csv/south_weekly_averages.csv')
central_data = pd.read_csv('./weather-csv/central_weekly_averages.csv')
east_data = pd.read_csv('./weather-csv/east_weekly_averages.csv')
fuel_prices = pd.read_csv('./fuel_prices.csv')
cabbage_prices = pd.read_csv('./vegetable-csv/Domestic_Cabbage.csv')
chinese_cabbage_prices = pd.read_csv('./vegetable-csv/Domestic_Chinese_cabbage.csv')
cauliflower_prices = pd.read_csv('./vegetable-csv/Domestic_Cauliflower.csv')



In [41]:
# 修改各地區天氣資料欄位名稱
north_data.columns = [f"north_{col}" if col != "週" else "週" for col in north_data.columns]
south_data.columns = [f"south_{col}" if col != "週" else "週" for col in south_data.columns]
central_data.columns = [f"central_{col}" if col != "週" else "週" for col in central_data.columns]
east_data.columns = [f"east_{col}" if col != "週" else "週" for col in east_data.columns]
print(south_data.head())

                       週  south_平均氣壓(hPa)  south_平均氣溫(℃)  south_平均相對溼度( %)  \
0  2018-12-31/2019-01-06          1019.90          19.53             89.83   
1  2019-01-07/2019-01-13          1016.39          20.49             86.57   
2  2019-01-14/2019-01-20          1016.59          18.90             79.86   
3  2019-01-21/2019-01-27          1017.86          16.84             78.29   
4  2019-01-28/2019-02-03          1017.10          19.08             78.75   

   south_平均風速(m/s)  south_累計雨量(mm)  south_累積日射量(MJ/m2)  
0             1.40            4.00                6.81  
1             1.36            1.93                8.40  
2             1.37            3.71                5.90  
3             1.16            0.64                5.67  
4             1.18            1.38               11.92  


In [46]:
# 合併各地區的天氣資料為單一 DataFrame
regional_data_1 = pd.merge(
    north_data,
    south_data,
    on='週',
    how='outer'
).merge(
    central_data,
    on='週',
    how='outer'
).merge(
    east_data,
    on='週',
    how='outer'
).rename(columns={"週": "date"})

regional_data_1 = regional_data_1.rename(columns={"週" : "date"})
print(regional_data_1.head())

# 檢查重複值
duplicates = regional_data_1[regional_data_1.duplicated(subset=['date'], keep=False)]
print("重複的記錄：")
print(duplicates)

                    date  north_平均氣壓(hPa)  north_平均氣溫(℃)  north_平均相對溼度( %)  \
0  2018-12-31/2019-01-06          1019.18          17.33             95.67   
1  2019-01-07/2019-01-13          1016.19          18.63             93.14   
2  2019-01-14/2019-01-20          1016.24          16.81             85.43   
3  2019-01-21/2019-01-27          1018.21          14.94             72.86   
4  2019-01-28/2019-02-03          1016.41          16.44             84.43   

   north_平均風速(m/s)  north_累計雨量(mm)  north_累積日射量(MJ/m2)  south_平均氣壓(hPa)  \
0             3.53            2.50                6.57          1019.90   
1             2.61            0.00                7.28          1016.39   
2             3.41            3.93                9.18          1016.59   
3             3.49            0.14               12.07          1017.86   
4             2.09            0.36               14.28          1017.10   

   south_平均氣溫(℃)  south_平均相對溼度( %)  ...  central_平均相對溼度( %)  \
0          19.53 

In [49]:
# 標準化欄位名稱
cabbage_prices.columns = ['date', 'cabbage_平均價', 'cabbage_交易量']
chinese_cabbage_prices.columns = ['date', 'chinese_cabbage_平均價', 'chinese_cabbage_交易量']
cauliflower_prices.columns = ['date', 'cauliflower_平均價', 'caulifower_交易量']

# 合併資料集
merged_vegetable_prices = pd.merge(
    cabbage_prices, 
    cauliflower_prices, 
    on='date', 
    how='outer'
).merge(
    chinese_cabbage_prices, 
    on='date', 
    how='outer'
)

print(merged_vegetable_prices.head())

# 檢查重複值
duplicates = merged_vegetable_prices[merged_vegetable_prices.duplicated(subset=['date'], keep=False)]
print("重複的記錄：")
print(duplicates)



                    date  cabbage_平均價  cabbage_交易量  cauliflower_平均價  \
0  2018-12-24/2018-12-30        26.60      3353.00              NaN   
1  2018-12-31/2019-01-06        22.22      4487.67            28.55   
2  2019-01-07/2019-01-13        20.40      4323.00            22.72   
3  2019-01-14/2019-01-20        19.78      5407.17            24.18   
4  2019-01-21/2019-01-27        20.47      6189.67            25.97   

   caulifower_交易量  chinese_cabbage_平均價  chinese_cabbage_交易量  
0             NaN                  NaN                  NaN  
1        39549.00                15.85              7315.50  
2        53975.33                10.63              7901.33  
3        37783.67                11.80              6472.67  
4        46511.33                14.42              7419.83  
重複的記錄：
Empty DataFrame
Columns: [date, cabbage_平均價, cabbage_交易量, cauliflower_平均價, caulifower_交易量, chinese_cabbage_平均價, chinese_cabbage_交易量]
Index: []


In [48]:
fuel_prices['Date'] = pd.to_datetime(fuel_prices['Date'], format='%Y/%m/%d')

# 建立新表格，填入時間範圍
expanded_rows = []
for _, row in fuel_prices.iterrows():
    start_date = row['Date']  # 當前日期
    end_date = start_date + timedelta(days=6)  # 往後延展七天的範圍
    date_range = f"{start_date.strftime('%Y-%m-%d')}/{end_date.strftime('%Y-%m-%d')}"  # 生成日期範圍字串
    
    expanded_rows.append({
        '週期': date_range,
        'Fuel_92': row['Fuel_92'],
        'Fuel_95': row['Fuel_95'],
        'Fuel_High': row['Fuel_High']
    })

# 將結果轉為新的 DataFrame
expanded_fuel_prices = pd.DataFrame(expanded_rows)
expanded_fuel_prices = expanded_fuel_prices.rename(columns={"週期" : "date"})

# 檢視結果
print(expanded_fuel_prices.head())

# 檢查重複值
duplicates = expanded_fuel_prices[expanded_fuel_prices.duplicated(subset=['date'], keep=False)]
print("重複的記錄：")
print(duplicates)



                    date  Fuel_92  Fuel_95  Fuel_High
0  2024-11-25/2024-12-01     28.6     30.1       27.2
1  2024-11-18/2024-11-24     28.6     30.1       27.1
2  2024-11-11/2024-11-17     28.4     29.9       26.8
3  2024-11-04/2024-11-10     28.8     30.3       27.2
4  2024-10-28/2024-11-03     28.6     30.1       27.0
重複的記錄：
Empty DataFrame
Columns: [date, Fuel_92, Fuel_95, Fuel_High]
Index: []


In [51]:
# 合併天氣數據與油價數據
weather_fuel_data = pd.merge(
    regional_data_1,
    expanded_fuel_prices,
    on='date',
    how='outer'
)

# 合併天氣與油價的結果再與蔬菜價格數據合併
complete_data = pd.merge(
    weather_fuel_data,
    merged_vegetable_prices,
    on='date',
    how='outer'
)

# 將日期範圍解析為起始日期
complete_data['start_date'] = complete_data['date'].str.split('/').str[0]
complete_data['start_date'] = pd.to_datetime(complete_data['start_date'], errors='coerce')

# 篩選出 2019 至 2022 年的數據
complete_data = complete_data.dropna(subset=['start_date'])  # 去掉無效日期
complete_data = complete_data[(complete_data['start_date'] >= pd.Timestamp('2019-01-01')) & (complete_data['start_date'] <= pd.Timestamp('2022-12-31'))]

# 檢視合併後的完整數據
print(complete_data.head())

# 檢查重複值
duplicates = complete_data[complete_data.duplicated(subset=['date'], keep=False)]
print("重複的記錄：")
print(duplicates)

complete_data.to_csv("./All_X_Y.csv", index=False)


                    date  north_平均氣壓(hPa)  north_平均氣溫(℃)  north_平均相對溼度( %)  \
2  2019-01-07/2019-01-13          1016.19          18.63             93.14   
3  2019-01-14/2019-01-20          1016.24          16.81             85.43   
4  2019-01-21/2019-01-27          1018.21          14.94             72.86   
5  2019-01-28/2019-02-03          1016.41          16.44             84.43   
6  2019-02-04/2019-02-10          1014.44          17.01             91.86   

   north_平均風速(m/s)  north_累計雨量(mm)  north_累積日射量(MJ/m2)  south_平均氣壓(hPa)  \
2             2.61            0.00                7.28          1016.39   
3             3.41            3.93                9.18          1016.59   
4             3.49            0.14               12.07          1017.86   
5             2.09            0.36               14.28          1017.10   
6             2.74            0.00                8.40              NaN   

   south_平均氣溫(℃)  south_平均相對溼度( %)  ...  Fuel_92  Fuel_95  Fuel_High  \
2       

In [30]:
# 檢查重複值
duplicates = complete_data[complete_data.duplicated(subset=['date'], keep=False)]
print("重複的記錄：")
print(duplicates)


重複的記錄：
                      date  平均氣壓(hPa)  平均氣溫(℃)  平均相對溼度( %)  平均風速(m/s)  \
5    2019-01-07/2019-01-13    1016.19    18.63       93.14       2.61   
6    2019-01-07/2019-01-13    1016.39    20.49       86.57       1.36   
7    2019-01-07/2019-01-13    1018.17    19.10       83.43       4.64   
8    2019-01-07/2019-01-13    1014.78    21.07       81.50       1.73   
9    2019-01-14/2019-01-20    1016.24    16.81       85.43       3.41   
..                     ...        ...      ...         ...        ...   
831  2022-12-19/2022-12-25    1013.37    14.21       72.71       1.71   
832  2022-12-26/2023-01-01    1018.60    16.47       78.29       6.27   
833  2022-12-26/2023-01-01    1019.63    17.80       81.57       0.97   
834  2022-12-26/2023-01-01    1020.49    16.77       75.29       5.56   
835  2022-12-26/2023-01-01    1017.16    16.53       80.86       1.64   

     累計雨量(mm)  累積日射量(MJ/m2)  Fuel_92  Fuel_95  Fuel_High  cabbage_平均價  \
5        0.00          7.28     25.0     26

In [22]:
# 檢查缺失值
print("缺失值統計：")
print(complete_data.isnull().sum())

# # 處理缺失值
# # 將數值型欄位缺失值填充為均值
# num_cols = complete_data.select_dtypes(include=['float64', 'int64']).columns
# complete_data[num_cols] = complete_data[num_cols].fillna(complete_data[num_cols].mean())

# # 將類別型欄位缺失值填充為 '未知'
# cat_cols = complete_data.select_dtypes(include=['object']).columns
# complete_data[cat_cols] = complete_data[cat_cols].fillna('未知')

# # 檢查處理後的缺失值
# print("處理後的缺失值統計：")
# print(complete_data.isnull().sum())

# # 如果不想填充，可以刪除包含缺失值的行（不推薦，僅當數據量充足時）
# # complete_data.dropna(inplace=True)

# # 儲存處理後的數據
# complete_data.to_csv('./complete_data_filled.csv', index=False)
# print("處理後的數據已儲存。")


缺失值統計：
date                     0
平均氣壓(hPa)                4
平均氣溫(℃)                  4
平均相對溼度( %)               4
平均風速(m/s)                4
累計雨量(mm)                 4
累積日射量(MJ/m2)             4
Fuel_92                138
Fuel_95                138
Fuel_High              138
cabbage_平均價              3
cabbage_交易量              3
cauliflower_平均價          2
caulifower_交易量           2
chinese_cabbage_平均價      2
chinese_cabbage_交易量      2
dtype: int64


In [13]:



# 檢查並重新命名日期欄位（如果需要）
def ensure_date_column(df, possible_names):
    for col in df.columns:
        if col in possible_names:
            df.rename(columns={col: 'date'}, inplace=True)
            break
    return df

# 將日期範圍轉換為開始日期
def parse_week_range_to_date(df, column_name):
    if column_name in df.columns:
        df['date'] = df[column_name].str.split('/').str[0]
        df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True)
    return df

# 針對需要的資料框重新命名並處理日期欄位
regional_data = ensure_date_column(regional_data_1, ['週', 'date'])
fuel_prices = ensure_date_column(fuel_prices, ['Date', 'date', '週', '日期'])
cabbage_prices = ensure_date_column(cabbage_prices, ['週', 'date'])
chinese_cabbage_prices = ensure_date_column(chinese_cabbage_prices, ['週'])
cauliflower_prices = ensure_date_column(cauliflower_prices, ['週', 'date'])

In [15]:
# 檢查所有資料框是否包含日期欄位
def check_date_column(df, name):
    if 'date' not in df.columns:
        raise KeyError(f"資料框 {name} 缺少 'date' 欄位。")

for name, df in zip(['regional_data', 'fuel_prices', 'cabbage_prices', 'chinese_cabbage_prices', 'cauliflower_prices'],
                    [regional_data, fuel_prices, cabbage_prices, chinese_cabbage_prices, cauliflower_prices]):
    check_date_column(df, name)

# 將日期欄位轉換為 datetime 格式
def parse_date(df, column_name):
    if column_name in df.columns:
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce', utc=True)
    return df

regional_data = parse_date(regional_data, 'date')
fuel_prices = parse_date(fuel_prices, 'date')
cabbage_prices = parse_date(cabbage_prices, 'date')
chinese_cabbage_prices = parse_date(chinese_cabbage_prices, 'date')
cauliflower_prices = parse_date(cauliflower_prices, 'date')

# 刪除缺失或無效的日期欄位的行
for df in [regional_data, fuel_prices, cabbage_prices, chinese_cabbage_prices, cauliflower_prices]:
    if 'date' in df.columns:
        df.dropna(subset=['date'], inplace=True)

# 確保日期欄位一致並為 datetime 格式
for df in [regional_data, fuel_prices, cabbage_prices, chinese_cabbage_prices, cauliflower_prices]:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date']).dt.normalize()

# 從日期中提取額外的特徵
def extract_date_features(df, date_column):
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    df['week'] = df[date_column].dt.isocalendar().week
    df['day_of_week'] = df[date_column].dt.dayofweek
    return df

cabbage_prices = extract_date_features(cabbage_prices, 'date')
fuel_prices = extract_date_features(fuel_prices, 'date')
chinese_cabbage_prices = extract_date_features(chinese_cabbage_prices, 'date')
cauliflower_prices = extract_date_features(cauliflower_prices, 'date')

# 合併所有資料為單一 DataFrame
try:
    merged_data = pd.merge(cabbage_prices, regional_data, on='date', how='left')
    merged_data = pd.merge(merged_data, chinese_cabbage_prices[['date', '平均價', '交易量']], on='date', how='left', suffixes=('', '_chinese_cabbage'))
    merged_data = pd.merge(merged_data, cauliflower_prices[['date', '平均價', '交易量']], on='date', how='left', suffixes=('', '_cauliflower'))
    merged_data = pd.merge(merged_data, fuel_prices, on=['year', 'month', 'week'], how='left')

except KeyError as e:
    raise KeyError(f"合併過程中出現錯誤: {e}. 請檢查所有資料框是否包含 'date' 欄位。")

# 根據分組計算均值並填補缺失值
merged_data['Fuel_92'] = merged_data.groupby('month')['Fuel_92'].transform(lambda x: x.fillna(x.mean()))
merged_data['Fuel_95'] = merged_data.groupby('month')['Fuel_95'].transform(lambda x: x.fillna(x.mean()))
merged_data['Fuel_High'] = merged_data.groupby('month')['Fuel_High'].transform(lambda x: x.fillna(x.mean()))

# 將合併後的 DataFrame 儲存為 CSV 文件
output_file = "All_X_Y.csv"
merged_data.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"合併後的資料已儲存至 {output_file}")

print(merged_data)


合併後的資料已儲存至 All_X_Y.csv
                       date_x    平均價       交易量  year  month  week  \
0   2018-12-24 00:00:00+00:00  26.60   3353.00  2018     12    52   
1   2018-12-31 00:00:00+00:00  22.22   4487.67  2018     12     1   
2   2019-01-07 00:00:00+00:00  20.40   4323.00  2019      1     2   
3   2019-01-14 00:00:00+00:00  19.78   5407.17  2019      1     3   
4   2019-01-21 00:00:00+00:00  20.47   6189.67  2019      1     4   
..                        ...    ...       ...   ...    ...   ...   
398 2024-11-18 00:00:00+00:00  45.30  16734.00  2024     11    47   
399 2024-11-25 00:00:00+00:00  40.70  17676.00  2024     11    48   
400 2024-11-25 00:00:00+00:00  40.70  17676.00  2024     11    48   
401 2024-11-25 00:00:00+00:00  40.70  17676.00  2024     11    48   
402 2024-11-25 00:00:00+00:00  40.70  17676.00  2024     11    48   

     day_of_week_x  平均氣壓(hPa)  平均氣溫(℃)  平均相對溼度( %)  ...  累積日射量(MJ/m2)  \
0                0        NaN      NaN         NaN  ...           NaN   
1 

In [ ]:
## 
''' 
Data Processing (2019~2022)

'''
## 

# Merge regional data into a single DataFrame
regional_data_1 = pd.concat([north_data, south_data, central_data, east_data], ignore_index=True)


# Check and rename date columns if necessary
def ensure_date_column(df, possible_names):
    for col in df.columns:
        if col in possible_names:
            df.rename(columns={col: 'date'}, inplace=True)
            break
    return df

# Rename the date columns where applicable
regional_data = ensure_date_column(regional_data_1, ['週', 'date'])
fuel_prices = ensure_date_column(fuel_prices, ['Date', 'date', '週', '日期'])
cabbage_prices = ensure_date_column(cabbage_prices, ['週', 'date'])

# Convert date columns to datetime
def parse_date(df, column_name):
    if column_name in df.columns:
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce', utc=True)
    return df

regional_data = parse_date(regional_data, 'date')
fuel_prices = parse_date(fuel_prices, 'date')
cabbage_prices = parse_date(cabbage_prices, 'date')

# Drop rows with missing or invalid 'date' values
for df in [regional_data, fuel_prices, cabbage_prices]:
    if 'date' in df.columns:
        df.dropna(subset=['date'], inplace=True)

# Ensure 'date' columns are consistent and datetimelike
for df in [regional_data, fuel_prices, cabbage_prices]:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date']).dt.normalize()

# Extract additional features from date
def extract_date_features(df, date_column):
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    df['week'] = df[date_column].dt.day // 7
    df['day_of_week'] = df[date_column].dt.dayofweek
    return df

cabbage_prices = extract_date_features(cabbage_prices, 'date')
fuel_prices = extract_date_features(fuel_prices, 'date')

# Merge all the data into a single DataFrame
try:
    merged_data = pd.merge(cabbage_prices, regional_data, on='date', how='left')
    merged_data = pd.merge(merged_data, fuel_prices, on=['year', 'month', 'week'], how='left')
except KeyError as e:
    raise KeyError(f"Error during merging: {e}. Please check that all dataframes contain a 'date' column.")

# 根據 Group 分組計算均值並填補缺失值
merged_data['Fuel_92'] = merged_data.groupby('month')['Fuel_92'].transform(lambda x: x.fillna(x.mean()))
merged_data['Fuel_95'] = merged_data.groupby('month')['Fuel_95'].transform(lambda x: x.fillna(x.mean()))
merged_data['Fuel_High'] = merged_data.groupby('month')['Fuel_High'].transform(lambda x: x.fillna(x.mean()))

# Save the merged DataFrame to a CSV file
output_file = "merged_data_m.csv"
merged_data.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Merged data saved to {output_file}")

print(merged_data)